# Interval-Based Time Series Regression

Interval-based regressors convert each series into a **vector of summary features**
computed over many random sub-intervals. A standard regressor (often tree-based)
then maps those features to a continuous target.


## Core idea
Sample intervals $[a, b]$ and compute statistics such as mean, std, slope, and min/max.
For series $x_{1:T}$, a feature map looks like:
\[\phi(x) = \{\text{mean}(x_{a:b}), \text{std}(x_{a:b}), \text{slope}(x_{a:b})\}_{(a,b)\in\mathcal{I}}\]
A regression model learns $y = g(\phi(x))$.


In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

rng = np.random.default_rng(7)
n_series = 60
length = 120
t = np.arange(length)

series = []
targets = []
for _ in range(n_series):
    amp = rng.uniform(0.8, 1.4)
    phase = rng.uniform(0, 2 * np.pi)
    noise = rng.normal(0, 0.2, size=length)
    x = amp * np.sin(2 * np.pi * t / 24 + phase) + noise
    series.append(x)
    targets.append(x[80:100].mean())

series = np.array(series)
targets = np.array(targets)

# Visualize one example with random intervals
example = series[0]
intervals = [(10, 30), (40, 70), (80, 100)]

fig = go.Figure()
fig.add_trace(go.Scatter(y=example, mode="lines", name="Series"))
for a, b in intervals:
    fig.add_vrect(x0=a, x1=b, fillcolor="LightSalmon", opacity=0.25, line_width=0)
fig.update_layout(title="Random intervals used for feature extraction", xaxis_title="Time index")
fig

## Interval features as predictors
Below we use just one interval mean as a feature to show how interval summaries can
capture target variation. Real models use **hundreds to thousands** of intervals.


In [ ]:
import plotly.graph_objects as go

feature = series[:, 80:100].mean(axis=1)
x = feature
y = targets

coef = np.polyfit(x, y, 1)
line = coef[0] * x + coef[1]

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode="markers", name="Series"))
fig.add_trace(go.Scatter(x=x, y=line, mode="lines", name="Fit"))
fig.update_layout(
    title="Interval mean vs target",
    xaxis_title="Mean on interval [80, 100]",
    yaxis_title="Target value",
)
fig

## sktime inventory for interval-based regressors
Find interval-family regressors in your local sktime registry. The naming varies across
versions, so we search by module/name keywords.


In [ ]:
try:
    import pandas as pd
    from sktime.registry import all_estimators

    ests = all_estimators(estimator_types="regressor", as_dataframe=True)
    mask = (
        ests["name"].str.contains("Interval|TimeSeriesForest|DrCIF", case=False, na=False)
        | ests["module"].str.contains("interval|forest|drcif", case=False, na=False)
    )
    print(ests.loc[mask, ["name", "module"]].sort_values("name").to_string(index=False))
except Exception as exc:
    print("sktime is not installed or registry lookup failed:", exc)


## Practical tips
- Increase interval count for accuracy, but monitor runtime.
- Use diverse statistics (mean, std, slope, quantiles) for richer features.
- Normalize per-series when amplitude varies widely.
